In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import sys
import time
import datetime
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score,average_precision_score
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold,StratifiedGroupKFold
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder
from torch.nn import TransformerDecoder
from torch.nn import LayerNorm
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.optim import lr_scheduler
from transformers import AdamW, get_linear_schedule_with_warmup
import gc
import random
import os
%matplotlib inline
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ===================
# Settings
# ===================
debug = False
ex = "175"
if not os.path.exists(f"../output/exp/ex{ex}"):
    os.makedirs(f"../output/exp/ex{ex}")
    os.makedirs(f"../output/exp/ex{ex}/ex{ex}_model")
logger_path = f"../output/exp/ex{ex}/ex_{ex}.txt"
model_path =f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}.pth"
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model config
batch_size = 24
n_epochs = 10
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

In [3]:
# =====================
# data
# =====================
id_path = f"../output/fe/fe047/fe047_id.parquet"
numerical_path = f"../output/fe/fe047/fe047_num_array.npy"
target_path = f"../output/fe/fe047/fe047_target_array.npy"
mask_path = f"../output/fe/fe047/fe047_mask_array.npy"
valid_path = f"../output/fe/fe047/fe047_valid_array.npy"
pred_use_path = f"../output/fe/fe047/fe047_pred_use_array.npy"

In [4]:
# =====================
# data
# =====================
pseudo_target = "073"
pseudo_id_path = f"../output/fe/fe061/fe061_id.parquet"
pseudo_numerical_path = f"../output/fe/fe061/fe061_num_array.npy"
pseudo_mask_path = f"../output/fe/fe061/fe061_mask_array.npy"
pseudo_valid_path = f"../output/fe/fe061/fe061_valid_array.npy"
pseudo_pred_use_path = f"../output/fe/fe061/fe061_pred_use_array.npy"

In [5]:
# ====================
# Function
# ====================
def preprocess(numerical_array, 
               mask_array,
               valid_array,
               ):
    
    attention_mask = mask_array == 0

    return {
        'input_data_numerical_array': numerical_array,
        'input_data_mask_array': mask_array,
        'input_data_valid_array': valid_array,
        'attention_mask': attention_mask,
    }

class FogDataset(Dataset):
    def __init__(self, numerical_array, 
                 mask_array,valid_array,
                 train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.valid_array = valid_array
        self.train = train
        self.y = y
    
    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],
            self.valid_array[item], 
            
        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array':torch.tensor(data['input_data_valid_array'], dtype=torch.long),   
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y":torch.tensor(self.y[item], dtype=torch.float32)
               }
        else:
            return {
             'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array':torch.tensor(data['input_data_valid_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

In [6]:
class FogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(FogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [7]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
setup_logger(out_file=logger_path)

2023-05-29 03:03:04,707 - INFO - logger set up


<RootLogger root (DEBUG)>

In [8]:
# ====================
# main
# ====================
df_id = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array = np.load(target_path)
mask_array = np.load(mask_path)
valid_array = np.load(valid_path)
pred_use_array = np.load(pred_use_path)

In [9]:
target1 = []
target2 = []
target3 = []
for i in range(len(target_array)):
    target1.append(np.sum(target_array[i,:,0]))
    target2.append(np.sum(target_array[i,:,1]))
    target3.append(np.sum(target_array[i,:,2]))

In [10]:
df_id["target1"] = target1
df_id["target2"] = target2
df_id["target3"] = target3
df_id["target1_1"] = df_id["target1"] > 0
df_id["target2_1"] = df_id["target2"] > 0
df_id["target3_1"] = df_id["target3"] > 0
df_id["target1_1"] = df_id["target1_1"].astype(np.int)
df_id["target2_1"] = df_id["target2_1"].astype(np.int)
df_id["target3_1"] = df_id["target3_1"].astype(np.int)

In [11]:
df_id["group"] = 0
df_id.loc[df_id["target1_1"] > 0,"group"] = 1
df_id.loc[df_id["target2_1"] > 0,"group"] = 2
df_id.loc[df_id["target3_1"] > 0,"group"] = 3

In [12]:
df_id["group"].value_counts()

0    4107
2     897
3     354
1      11
Name: group, dtype: int64

In [13]:
pseudo_numerical_array = np.load(pseudo_numerical_path)
pseudo_mask_array = np.load(pseudo_mask_path)
pseudo_valid_array = np.load(pseudo_valid_path)
pseudo_pred_use_array = np.load(pseudo_pred_use_path)

In [14]:
with timer("gru"):
    set_seed(seed)
    y_oof = np.empty([len(target_array),5000,3])
    gkf = StratifiedGroupKFold(n_splits=n_splits,shuffle=True,random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                             y = df_id["group"].values,
                                                             groups=df_id["subject"].values)):
        LOGGER.info(f"start fold:{fold}")
        with timer(f"fold {fold}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]
            
            pseudo_target_array = np.load(f"../output/fe/fe{pseudo_target}/fe{pseudo_target}_target_array_{fold}.npy")
            train_numerical_array = np.concatenate([train_numerical_array,pseudo_numerical_array],axis=0)
            train_target_array = np.concatenate([train_target_array,pseudo_target_array],axis=0)
            train_mask_array = np.concatenate([train_mask_array,pseudo_mask_array],axis=0)
            train_valid_array = np.concatenate([train_valid_array,pseudo_valid_array],axis=0)
            
            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array, 
                                train=True,
                                y=train_target_array)
            
            val_ = FogDataset(val_numerical_array,
                                val_mask_array,
                                val_valid_array, 
                                train=True,
                                y=val_target_array)
            
            
            train_loader = DataLoader(dataset=train_, 
                                  batch_size=batch_size, 
                                  shuffle = True , num_workers=8)
            val_loader = DataLoader(dataset=val_, 
                                batch_size=batch_size, 
                                shuffle = False , num_workers=8)
            
            model = FogRnnModel()
            model = model.to(device)
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))
                tk0 = tqdm(train_loader, total=len(train_loader))
                for d in tk0:
                    # =========================
                    # data loader
                    # =========================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array = d['input_data_mask_array'].to(device)
                    input_data_valid_array = d['input_data_valid_array'].to(device)
                    attention_mask = d['attention_mask'].to(device)
                    y = d["y"].to(device)
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[(input_data_valid_array == 1)], y[input_data_valid_array == 1])
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
            
                # ==========================
                # eval
                # ==========================
                model.eval()  # switch model to the evaluation mode
                val_preds = np.ndarray((0,5000,3))
                tk0 = tqdm(val_loader, total=len(val_loader))
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array = d['input_data_mask_array'].to(device)
                        attention_mask = d['attention_mask'].to(device)
                        output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:,0],
                                                          val_preds[pred_valid_index][:,0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:,1],
                                                          val_preds[pred_valid_index][:,1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:,2],
                                                          val_preds[pred_valid_index][:,2])
                map_score = np.mean([Turn,
                                     Walking])
                LOGGER.info(f"fold:{fold} epoch : {epoch},train loss {train_loss} map:{map_score} starthesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                #best_val_preds = val_preds.copy()
                #torch.save(model.state_dict(), f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth") 
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth") 
            y_oof[valid_idx] = best_val_preds
    np.save(f"../output/exp/ex{ex}/ex{ex}_oof.npy",y_oof)

2023-05-29 03:03:11,412 - INFO - start fold:0


  0%|          | 0/344 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

2023-05-29 03:06:40,821 - INFO - fold:0 epoch : 0,train loss 0.15210964811186112 map:0.2051529048063937 starthesi:7.608089264871e-05 turn:0.4028233088911223 walking :0.00748250072166513


save weight


...

  0%|          | 0/44 [00:00<?, ?it/s]

2023-05-29 05:51:46,013 - INFO - fold:4 epoch : 8,train loss 0.047176330425843065 map:0.32329059476297717 starthesi:3.956422725304519e-05 turn:0.2030110857780666 walking :0.4435701037478877


  0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

2023-05-29 05:55:14,672 - INFO - fold:4 epoch : 9,train loss 0.045033946971275975 map:0.33020404580771734 starthesi:3.957796223689709e-05 turn:0.20827285305474186 walking :0.4521352385606928
2023-05-29 05:55:14,708 - INFO - [fold 4] done in 2081 s
2023-05-29 05:55:15,134 - INFO - [gru] done in 10324 s


In [15]:
val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
StartHesitation = average_precision_score(target_array[val_pred_index][:,0],
                                                          y_oof[val_pred_index ][:,0])
Turn = average_precision_score(target_array[val_pred_index ][:,1],
                                                          y_oof[val_pred_index ][:,1])
Walking = average_precision_score(target_array[val_pred_index ][:,2],
                                                          y_oof[val_pred_index ][:,2])

map_score = np.mean([StartHesitation,
                               Turn,
                               Walking])
LOGGER.info(f"cv map:{map_score} starthesi:{StartHesitation} turn:{Turn} walking :{Walking}")

2023-05-29 05:55:21,187 - INFO - cv map:0.29171275544936953 starthesi:0.0001968713147668923 turn:0.5806698012157918 walking :0.29427159381754997


In [16]:
import json
kaggle_json = {
  "title": f"fog-ex{ex}",
  "id": f"takoihiraokazu/fog-ex{ex}",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

with open(f"../output/exp/ex{ex}/ex{ex}_model/dataset-metadata.json", 'w') as f:
    json.dump(kaggle_json, f)